In [ ]:
import re
# download data from from Google Drive
!gdown 1uml6lt5diPDjBnJYVECBrafVJEISRU2-

In [ ]:
# read the file in to a list of tokens
from collections import defaultdict
noun_tokens=defaultdict(int)
verb_tokens=defaultdict(int)
tokens=[]
filename="Manchester-morph+changes_utterancesremoved.cha"
with open(filename) as file:
        for line in file:
            if re.match("^\*MOT",line):
                line=re.sub('\*[A-Z]+\:','', line)
                line=re.sub(' \.',' ', line)
                line=re.sub(' \?',' ', line)
                line=re.sub(' \,',' ', line)
                line=re.sub(' \!',' ', line)
                line = "{ " + line + " } "
                tokens.extend(line.split())

            if re.match("^\%mor",line):
                line=re.sub('\%mor\:','', line)
                mortokens = line.split()
                for tok in mortokens:
                     if re.match("^noun\|",tok):
                        tok=re.sub("^noun\|(.+)",'\\1', tok)
                        noun_tokens[str.lower(tok)] += 1
                     elif re.match("^verb\|",tok):
                        tok=re.sub("^verb\|(.+)",'\\1', tok)
                        verb_tokens[str.lower(tok)] += 1


In [ ]:
sorted_noun_counts=sorted(noun_tokens.items(), key=lambda item: item[1], reverse=True)
sorted_verb_counts=sorted(verb_tokens.items(), key=lambda item: item[1], reverse=True)
# Extract the list of tokens, by transposing the list of lists so that there is a list of tokens a list of counts and then just selecting the former
sorted_noun_tokens=list(zip(*sorted_noun_counts))[0]
sorted_verb_tokens=list(zip(*sorted_verb_counts))[0]
excluded_nouns=["mummy","daddy","john","carl","dominic"] # proper names
excluded_verbs=[""]
sorted_noun_tokens=[x for x in sorted_noun_tokens if x not in excluded_nouns]
sorted_noun_tokens=[x for x in sorted_noun_tokens if noun_tokens[x] > verb_tokens[x]]
sorted_verb_tokens=[x for x in sorted_verb_tokens if x not in excluded_verbs]
sorted_verb_tokens=[x for x in sorted_verb_tokens if verb_tokens[x] > noun_tokens[x]]
seed_set_size = 10
noun_seeds=sorted_noun_tokens[0:seed_set_size]
verb_seeds=sorted_verb_tokens[0:seed_set_size]
type_list=list(noun_seeds)
type_list.extend(verb_seeds)
tokens.insert(0,"{")
tokens.insert(len(tokens),"}")
token_count=len(tokens)





In [ ]:
from collections import defaultdict
import numpy as np
type_count = len(type_list)
targets = []
positions = []
contextwordcounts = []
d = defaultdict(lambda: [0] * (seed_set_size*2))
window_size = 2
#M = np.zeros((type_count, 4, type_count))
toks_to_ignore = ["{","}"]

for i, word in enumerate(tokens):
            # Find the index in the tokenized sentence vector for the beginning of the window (the current token minus window size or zero whichever is the lower)
            begin = max(i - window_size, 0)
            # Find the index in the tokenized sentence vector for the end of the window (the current token plus window size or the length of the sentence whichever is the lower)
            end  = min(i + window_size, token_count)
            # Extract the text from beginning of window to the end
            context = tokens[begin: end + 1]
            # Remove the target word from its own window
            context.remove(tokens[i])
            context=["noun" if noun_seeds.count(w) > 0 else w for w in context]
            context=["verb" if verb_seeds.count(w) > 0 else w for w in context]
            if type_list.count(word) > 0:
                target_id = type_list.index(word)
            #if toks_to_ignore.count(word) == 0:
                p1 = context[1] + "_X_" + context[2]
                p1 = re.sub(".+(\{.+)","\\1",p1)
                p1 = re.sub("(.+\}).+","\\1",p1)

                p1a = context[1] + "_X_"
                p1a = re.sub(".+(\{.+)","\\1",p1a)
                p1a = re.sub("(.+\}).+","\\1",p1a)

                p2 = "X_" + context[2] + "_" + context[3]
                p2 = re.sub(".+(\{.+)","\\1",p2)
                p2 = re.sub("(.+\}).+","\\1",p2)

                p2a = "X_" + context[2]
                p2a = re.sub(".+(\{.+)","\\1",p2a)
                p2a = re.sub("(.+\}).+","\\1",p2a)

                p3 = context[0] + "_" + context[1] + "_X"
                p3 = re.sub(".+(\{.+)","\\1",p3)
                p3 = re.sub("(.+\}).+","\\1",p3)

                # Find the ID and hence the list index for the current token
                #token_id = type_list.index(token)
                # Add 1 to the current context dimension for the current target word
                d[p1][target_id] += 1
                d[p2][target_id] += 1
                d[p3][target_id] += 1
                d[p1a][target_id] += 1
                d[p2a][target_id] += 1

                #M[current_row][position][token_id] += 1



In [ ]:
# Convert to a hierachical data frame
import pandas as pd
df=pd.DataFrame(d)
df.index = type_list

In [ ]:
df

In [ ]:
from collections import defaultdict
import numpy as np
window_size = 2
toks_to_ignore = ["{","}"]

for i, word in enumerate(tokens):
            # Find the index in the tokenized sentence vector for the beginning of the window (the current token minus window size or zero whichever is the lower)
            begin = max(i - window_size, 0)
            # Find the index in the tokenized sentence vector for the end of the window (the current token plus window size or the length of the sentence whichever is the lower)
            end  = min(i + window_size, token_count)
            # Extract the text from beginning of window to the end
            context = tokens[begin: end + 1]
            # Remove the target word from its own window
            context.remove(tokens[i])
            context=["noun" if noun_seeds.count(w) > 0 else w for w in context]
            context=["verb" if verb_seeds.count(w) > 0 else w for w in context]

            if toks_to_ignore.count(word) == 0:
                p1 = context[1] + "_X_" + context[2]
                p1 = re.sub(".+(\{.+)","\\1",p1)
                p1 = re.sub("(.+\}).+","\\1",p1)
                p1a = context[1] + "_X"
                p1a = re.sub(".+(\{.+)","\\1",p1a)
                p1a = re.sub("(.+\}).+","\\1",p1a)
                if list(df.columns).count(p1) > 0:
                    #print(p1)
                    #print(df.get(p1))
                    #print(df.get(p1).idxmax())
                    if sorted_noun_tokens.count(df.get(p1).idxmax()) > 0:
                        category="noun"
                    if sorted_verb_tokens.count(df.get(p1).idxmax()) > 0:
                        category="verb"
                elif list(df.columns).count(p1a) > 0:
                    if sorted_noun_tokens.count(df.get(p1a).idxmax()) > 0:
                        category="noun"
                    if sorted_verb_tokens.count(df.get(p1a).idxmax()) > 0:
                        category="verb"
                else:
                    category = "other"
                print(word + " " + category)
